In [2]:
import numpy as np
np.set_printoptions(threshold=np.nan, linewidth=np.nan)

In [3]:
import pickle
with open('df.pickle', mode='rb') as h:
    _, df_test = pickle.load(h)
with open('hex.pickle', mode='rb') as h:
    hex_data = pickle.load(h)
labels = df_test['label']
name_id = hex_data['name_id']
id_hls = map(set, hex_data['id_hierarchical_labels'])  # id_hls for id_hierarchical_labels
id_fh = map(lambda x: max(x, key=len), id_hls)  # id_fh for id_full_hierarchy

In [14]:
def get_accuracy(Y):
    """
    Type 0: Is the full hierarchy
    Type 1: Covers the full hierarchy
    Type 2: Is any partial hierarchy
    Type 3: Covers any partial hierarchy
    Implication: 0->1, 0->2, 1->3, 2->3
    """
    accuracy = np.zeros(4, dtype=np.float32)
    for i in range(0, len(Y)):
        y = Y[i]
        y = np.nonzero((y / y.max()) > 0.005)[0]  # designed for probs, also fine for binary
        lid = name_id[labels[i]]
        if tuple(y) == id_fh[lid]:
            accuracy += 1
        elif set(id_fh[lid]) in set(y):
            accuracy[[1, 3]] += 1
        elif tuple(y) in id_hls[lid]:
            accuracy[[2, 3]] += 1
        elif any(set(x) in set(y) for x in id_hls[lid]):
            accuracy[3] += 1
    return accuracy / len(Y)

In [15]:
# caffe accuracy
from os import listdir
import re
models = listdir('../caffemodels/')
iters = [int(re.findall('\d+', x)[0]) for x in models]
iters, models = zip(*sorted(zip(iters, models), key=lambda x: x[0]))
iters

(10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000)

In [16]:
iter_Y = np.load('../test_caffe.npy')
np.array(map(get_accuracy, iter_Y), dtype=np.float32)

array([[ 0.        ,  0.        ,  0.32185274,  0.32185274],
       [ 0.        ,  0.        ,  0.27078384,  0.27078384],
       [ 0.        ,  0.        ,  0.27019003,  0.27019003],
       [ 0.        ,  0.        ,  0.25534442,  0.25534442],
       [ 0.        ,  0.        ,  0.26128265,  0.26128265],
       [ 0.        ,  0.        ,  0.26187649,  0.26187649],
       [ 0.        ,  0.        ,  0.26187649,  0.26187649],
       [ 0.        ,  0.        ,  0.26306415,  0.26306415],
       [ 0.        ,  0.        ,  0.26306415,  0.26306415],
       [ 0.        ,  0.        ,  0.26306415,  0.26306415]], dtype=float32)

In [4]:
# svm accuracy
kernels = ['linear', 'poly', 'rbf']
kernel_Y = np.load('../test_svm.npy')

In [8]:
np.array(map(get_accuracy, kernel_Y), dtype=np.float32)

array([[ 0.45486936,  0.45486936,  0.83313537,  0.83313537],
       [ 0.36520189,  0.36520189,  0.93052256,  0.93052256],
       [ 0.28325415,  0.28325415,  0.92399049,  0.92399049],
       [ 0.        ,  0.        ,  0.        ,  0.        ]], dtype=float32)

In [18]:
get_accuracy(np.load('../test_crf.npy'))

array([ 0.03450609,  0.03450609,  0.14140731,  0.14140731], dtype=float32)